In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import solve_bvp

Helper Functions

In [ ]:
def A(t):
    pass
    
def B(t):
    # Draw initial values from uniform distribution
    values = np.random.uniform(0, 1, size=1000) 
    # Determine whether values are positive or negative randomly
    eps = values * np.random.choice([-1, 1], size=1000)

    if 0 <= t <= 2.5 + eps: # Baby asleep
        return 1
    elif 2.5 + eps < t <= 3.5 + eps: # Baby awake
        return 0
    
    return False # Case not handled by the current logic
    
def delta_t(t):
    if 8 <= t <= 24: # Nightime hours
        return 0.9
    elif 0 < t < 8: # Daytime hours
        return 0.1
    
    return False # Case not handled by the current logic

    
u = lambda t: '?' 
Dp = lambda t: '?' # Function in relation to baby sleep???
Np = lambda t: '?' # "
K = lambda t: '?'
W = lambda t: np.random.normal(7, 9) # Work hours vary between 7-9
Z = lambda t: np.random.normal(-5, 5) # High variance (number chosen is arbitrary, feel free to adjust)

X_prime = lambda t: (1 - delta_t(t)) * u(t)

alpha = '?' # Tunable parameter

Cost Functional

In [ ]:
def functional(t, y):
    # THIS NEEDS TO BE MODIFIED TO WORK WITH SOLVE_BVP
    return B(t) * Dp(t)**2/(K*Np(t)) - W(t) + alpha*Z(t)

def bc():
    '''
    Boundary Conditions:
    
    X(t0) = X(tf) = a
    '''
    pass

Solve the ODE

In [ ]:
t0, tf = 0, 24

t_span = (t0, tf)
t = np.linspace(t0, tf, 200)
sol = solve_bvp(functional, t_span, y0, t_eval=t, args=())